# Code for the Retailer Game (Homework Problems)

### Benchmark case

In [36]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

#the vector of prices
price = np.array([60, 54, 48, 36])
#the vector of demands
demand = np.array([125, 162.5, 217.5, 348.8])
#salvage value
s = 25
#total number of inventory
I = 2000
#Time horizon
T = 15
#full price week
full_price_week = 1

#number of price levels
N = len(price)

In [37]:
#########Model Set-up Using Function###############
def model_setup():
    
    m = Model("Retail")
    
    # number of weeks to offer price level i
    x = m.addVars(N, name = "x")

    # set objective
    m.setObjective( quicksum(price[i]*demand[i]*x[i] for i in range(N)) + s*(I - quicksum(demand[i]*x[i] for i in range(N))), GRB.MAXIMIZE)

    # capcity constraint: 
    m.addConstr( quicksum(demand[i]*x[i] for i in range(N)) <= I , "capacity")

    # time constraint: 
    m.addConstr( quicksum(x[i] for i in range(N)) <= T , "time")

    # full price constraint: 
    m.addConstr( x[0] >= full_price_week , "full_price")
    
    return m

In [38]:
# setup the model
m = model_setup()

# Solving the model
m.optimize()

# With function model_setup(). the decision variables x become local variables and cannot be accessed directly
x = np.zeros(N)

#  Print and store optimal solutions 
i = 0
print("\n Optimal solution:")
for v in m.getVars():
    x[i] = v.x
    print(v.VarName, v.x)
    i = i + 1
      
# Print optimal value
print("\n Optimal profit:")
print('Obj:', m.objVal)


Optimize a model with 3 rows, 4 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [4e+03, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+05   1.587200e+01   0.000000e+00      0s
       2    1.1675000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.167500000e+05

 Optimal solution:
x[0] 11.666666666666666
x[1] 3.333333333333334
x[2] 0.0
x[3] 0.0

 Optimal profit:
Obj: 116750.0


### Adjustment of markdown strategy when there were only 25 units sold in the first week

In [39]:
#acutal sales in the first week
sales = 25

#compute the revenue under the original markdown policy
revenue_static = (  price[0]*sales + price[0]*demand[0]*(x[0] - 1) + 
                 sum(price[i]*demand[i]*x[i] for i in range(1,N)) + 
                 s*(I - sales - demand[0]*(x[0] - 1) - sum(demand[i]*x[i] for i in range(1,N)))  )

print("\n The revenue without adjusting the markdown policy is:",  revenue_static)   


 The revenue without adjusting the markdown policy is: 113250.0


In [40]:
#adjusting the markdown policy by reoptimize the remaining 14 weeks
#horizon reduces by 1
T = T - 1
#inventory reduces by sales
I = max(I - sales, 0)
#the number of weeks required for full price reduces by 1
full_price_week = full_price_week - 1

# setup the model
m = model_setup()

# Solving the model
m.optimize()

# the adjusted pricing decisions
x_adjusted = np.zeros(N)

# store optimal solutions 
i = 0
for v in m.getVars():
    x_adjusted[i] = v.x
    i = i + 1

print("\n The adjusted pricing decision is:",  x_adjusted)       

revenue_adjusted = price[0]*sales + m.objVal

print("\n The revenue from adjusting the markdown policy is:",  revenue_adjusted) 

ratio = (revenue_adjusted - revenue_static)/revenue_static*100
print('\n Percentage improvement is %g percent' %  ratio) 

Optimize a model with 3 rows, 4 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [4e+03, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1850000e+05   2.868000e+01   0.000000e+00      0s
       2    1.1265000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  1.126500000e+05

 The adjusted pricing decision is: [8. 6. 0. 0.]

 The revenue from adjusting the markdown policy is: 114150.0

 Percentage improvement is 0.794702 percent


### Using finer price grids

In [42]:
#########Reset parameters############

#the vector of prices
price = np.array([60, 58, 56, 54, 52, 50, 48, 46, 44, 42, 40, 38, 36])
#the vector of demands
demand = np.array([125, 137.5, 150, 162.5, 180.8, 199.1, 217.5, 239.4, 261.3, 283.2, 305.1, 327, 348.8])
#salvage value
s = 25
#total number of inventory
I = 2000
#Time horizon
T = 15
#full price week
full_price_week = 1

#number of price levels
N = len(price)

# reset the model
m = model_setup()

# Solving the model
m.optimize()

#  Print optimal solutions 
print("\n Optimal solution:")
for v in m.getVars():
    print(v.VarName, v.x)
      
# Print optimal value
print("\n Optimal profit:")
print('Obj:', m.objVal)

Optimize a model with 3 rows, 13 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [4e+03, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 13 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+05   1.587200e+01   0.000000e+00      0s
       2    1.1725000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.172500000e+05

 Optimal solution:
x[0] 5.0
x[1] 10.0
x[2] 0.0
x[3] 0.0
x[4] 0.0
x[5] 0.0
x[6] 0.0
x[7] 0.0
x[8] 0.0
x[9] 0.0
x[10] 0.0
x[11] 0.0
x[12] 0.0

 Optimal profit:
Obj: 117250.0
